Import packages and libraries

In [1]:
import tensorflow as tf
import os
import keras
import numpy as np
import easygdf
import mne
import json
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score , precision_score , recall_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, MaxPooling1D, Dropout, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import get_data
from earlystopclass import EarlyStopAtAccuracy
from ipynb.fs.full.data_preprocess import save_data_in_csv,get_all_info,get_event_position,save_data_in_array,combine_data_labels,interpolation,cropping,events_to_classes
from sklearn.model_selection import train_test_split
from colabcode import ColabCode
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.model_selection import KFold


The network model

In [2]:
# Define the squeeze excitation block function
def squeeze_excitation_block(inputs, r):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling1D()(inputs)
    se = layers.Dense(filters // r, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1, filters))(se)
    out=inputs * se
    return out

# def SqueezeAndExcitation(inputs, ratio=8): 
#     b, _, _, c = inputs.shape 
#     x = GlobalAveragePooling2D()(inputs) 
#     x = Dense(c//ratio, activation="relu", use_bias=False)(x) 
#     x = Dense(c, activation="sigmoid", use_bias=False)(x) 
#     x = inputs * x 
#     return x 

# Set random seed for reproducibility
#tf.random.set_seed(1234)


def create_model():
    # Define input shape
    input_shape = (1000,22)

    # Define input layers for each branch
    input_1 = tf.keras.Input(shape=input_shape)
    input_2 = tf.keras.Input(shape=input_shape)
    input_3 = tf.keras.Input(shape=input_shape)

    # First Branch
    x1 = tf.keras.layers.Conv1D(32, 20, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)
    x1 = squeeze_excitation_block(x1, r=8)
    x1 = tf.keras.layers.MaxPooling1D()(x1)


    x1 = tf.keras.layers.Conv1D(32, 9, strides=2, padding='same', kernel_regularizer=l2(0.01))(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)
    x1 = squeeze_excitation_block(x1, r=8)
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    skip_connection1 = x1 ############

    x1 = tf.keras.layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)

    x1 = tf.keras.layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)
    x1 = squeeze_excitation_block(x1, r=8)
    x1 = tf.keras.layers.add([x1, skip_connection1])############
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    skip_connection2 = x1 ############

    x1 = tf.keras.layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)

    x1 = tf.keras.layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x1)
    x1 = tf.keras.layers.BatchNormalization()(x1)
    x1 = tf.keras.layers.ELU()(x1)
    x1 = squeeze_excitation_block(x1, r=8)
    x1 = tf.keras.layers.add([x1, skip_connection2])############
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    x1 = tf.keras.layers.Dropout(0.5)(x1)

    x1 = tf.keras.layers.Flatten()(x1)

    # Second Branch
    x2 = tf.keras.layers.Conv1D(32, 13, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ELU()(x2)
    x2 = squeeze_excitation_block(x2, r=8)

    x2 = tf.keras.layers.Conv1D(32, 7, strides=2, padding='same', kernel_regularizer=l2(0.01))(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ELU()(x2)
    x2 = squeeze_excitation_block(x2, r=8)
    x2 = tf.keras.layers.MaxPooling1D()(x2)

    x2 = tf.keras.layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ELU()(x2)
    x2 = squeeze_excitation_block(x2, r=8)
    x2 = tf.keras.layers.MaxPooling1D()(x2)
    skip_connection3 = x2

    x2 = tf.keras.layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ELU()(x2)

    x2 = tf.keras.layers.Conv1D(32, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x2)
    x2 = tf.keras.layers.BatchNormalization()(x2)
    x2 = tf.keras.layers.ELU()(x2)
    x2 = squeeze_excitation_block(x2, r=8)
    x2 = tf.keras.layers.add([x2, skip_connection3])############
    x2 = tf.keras.layers.MaxPooling1D()(x2)

    x2 = tf.keras.layers.Dropout(0.5)(x2)

    x2 = tf.keras.layers.Flatten()(x2)

    # Third Branch
    x3 = tf.keras.layers.Conv1D(32, 15, strides=2, padding='same', kernel_regularizer=l2(0.01))(input_3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.ELU()(x3)
    x3 = squeeze_excitation_block(x3, r=8)
    x3 = tf.keras.layers.MaxPooling1D()(x3)

    x3 = tf.keras.layers.Conv1D(32, 10, strides=2, padding='same', kernel_regularizer=l2(0.01))(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.ELU()(x3)
    x3 = squeeze_excitation_block(x3, r=8)

    x3 = tf.keras.layers.Conv1D(32, 5, strides=1, padding='same', kernel_regularizer=l2(0.01))(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.ELU()(x3)
    x3 = squeeze_excitation_block(x3, r=8)
    x3 = tf.keras.layers.MaxPooling1D()(x3)

    x3 = tf.keras.layers.Conv1D(48, 3, strides=1, padding='same', kernel_regularizer=l2(0.01))(x3)
    x3 = tf.keras.layers.BatchNormalization()(x3)
    x3 = tf.keras.layers.ELU()(x3)
    x3 = squeeze_excitation_block(x3, r=8)
    x3 = tf.keras.layers.MaxPooling1D()(x3)

    x3 = tf.keras.layers.Dropout(0.5)(x3)

    x3 = tf.keras.layers.Flatten()(x3)

    # Concatenate the outputs of three branches
    merged = tf.keras.layers.concatenate([x1, x2, x3])


    #fc1 = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(merged)
    #fc2 = tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(fc1)

    # Dropout layer
    x = tf.keras.layers.Dropout(0.5)(merged)

    # Output layer
    output = tf.keras.layers.Dense(4, activation='softmax')(x)

    # Create model
    model = Model(inputs=[input_1, input_2, input_3], outputs=output)

    return model


# Print model summary
#model.summary()

cropping for test data

In [3]:
def test_cropping(interpolated_x_test,old_x_test):
    # Initialize empty lists to store cropped segments and corresponding labels
    increased_x_test = []
    # Iterate over each sample
    for sample_idx in range(interpolated_x_test.shape[0]):
        # Get the EEG data for the current sample
        sample_data = interpolated_x_test[sample_idx, :, :]
        # Calculate the number of segments
        num_segments = (sample_data.shape[0] - 1000) // 10 + 1

        temp=[]
        temp.append(old_x_test[sample_idx])
        # Iterate over each segment
        for segment_idx in range(num_segments):
            # Calculate start index of the segment
            start_idx = segment_idx * 10
            # Extract the segment
            segment = sample_data[start_idx:start_idx + 1000, :]
            # Append the segment to the list of cropped segments
            temp.append(segment)

        increased_x_test.append(temp)
    # Convert the lists to numpy arrays
    increased_x_test = np.array(increased_x_test)
    

    return increased_x_test

K fold

In [4]:
subjects=["A01T","A02T", "A03T","A04T","A05T","A06T","A07T","A08T","A09T"]


accuracies=[]
f1_scores=[]
recalls=[]
precisions=[]

for subject in subjects:

    filename_temp=subject+".gdf"
    filename=[filename_temp]
    
    label=[subject]

    #get info of events' positions
    positions_of_labels=get_event_position(filename,label)

    #save raw eeg data recorded from subjects in array
    data_arrays=save_data_in_array(filename,label)

    #combine data features or samples with their actual labels
    combined_data=combine_data_labels(data_arrays,positions_of_labels,label)

    #Generate shuffled indices
    indices = np.arange(len(combined_data))
    np.random.shuffle(indices)

    # Shuffle x_train and y_train using the shuffled indices
    shuffled_combined_data = [combined_data[i] for i in indices]

    # Convert to numpy array of object type
    shuffled_combined_data = np.array(shuffled_combined_data, dtype=object)

    kth_accuracies = []
    kth_f1_scores = []
    kth_recall = []
    kth_precision = []

    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=1)

    for train_index, test_index in kf.split(shuffled_combined_data):

        model = create_model()
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Split the data into train and test sets
        raw_train, raw_test = shuffled_combined_data[train_index], shuffled_combined_data[test_index]

        #seprate labels from feature data
        raw_x_train,raw_y_train = zip(*raw_train)
        x_test,y_test = zip(*raw_test)

        #convert type of array to numpy array
        raw_x_train=np.array(raw_x_train)
        raw_y_train=np.array(raw_y_train)

        x_test=np.array(x_test)
        y_test=np.array(y_test)

        #change shape of y_test and y_train beacuse this is better shape more practical and 
        raw_y_train = raw_y_train.reshape(-1, 1)
        y_test = y_test.reshape(-1, 1)

        #perform interpolation operation and increase the size of matrices of each samples
        interpolated_raw_x_train=interpolation(raw_x_train)

        #perform cropping operation to increase the size of the data by 6 times
        cropped_segments,segment_labels=cropping(interpolated_raw_x_train,raw_y_train)

        #concatenate new created data from cropping operation to primary data
        x_train = np.concatenate((raw_x_train, cropped_segments), axis=0)
        y_train = np.concatenate((raw_y_train, segment_labels), axis=0)


        # shuffle x_train and y_train using the shuffled indices
        new_indices = np.arange(len(x_train))
        np.random.shuffle(new_indices)

        x_train = x_train[new_indices]
        y_train = y_train[new_indices]

        #cahnge events' number to 0-3 classes for y test/train
        y_test=events_to_classes(y_test)
        y_train=events_to_classes(y_train)

        # Split the training data into training and cross-validation sets
        x_train, x_cv , y_train, y_cv =train_test_split(x_train, y_train, test_size=0.10, random_state=1)


        
        checkpoint = ModelCheckpoint("model_checkpoint.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        #early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
        early_stopping = EarlyStopAtAccuracy(monitor='val_accuracy', target_accuracy=0.93, patience=0, verbose=1, mode='max')

        # Train the model
        history = model.fit(
            [x_train, x_train, x_train], y_train,
            epochs=100,
            batch_size=30,
            validation_data=([x_cv, x_cv, x_cv], y_cv),
            callbacks=[early_stopping]
        )
        
        original_x_test=x_test
        original_y_test=y_test

        interpolated_x_test=interpolation(x_test)
        increased_x_test=test_cropping(interpolated_x_test,original_x_test)

        final_y_pred=[]
        for i in range(increased_x_test.shape[0]):
            y_predict=model.predict([increased_x_test[i], increased_x_test[i], increased_x_test[i]])
            y_pred_labels = np.argmax(y_predict, axis=1)
            counter = Counter(y_pred_labels)
            most_common = counter.most_common(1)
            final_y_pred.append(most_common[0][0])
            
        final_y_pred=np.array(final_y_pred)##
        final_y_pred=final_y_pred.reshape(-1,1)
        final_y_pred = final_y_pred.astype(np.int32)    
           
        f1 = f1_score(y_test, final_y_pred, average='weighted')
        test_accuracy = accuracy_score(y_test, final_y_pred)
        precision = precision_score(y_test, final_y_pred, average='weighted')
        recall = recall_score(y_test, final_y_pred, average='weighted')

        kth_accuracies.append(test_accuracy)
        kth_f1_scores.append(f1)
        kth_precision.append(precision)
        kth_recall.append(recall)

    kth_accuracies=np.array(kth_accuracies)
    kth_f1_scores=np.array(kth_f1_scores) 
    kth_recall=np.array(kth_recall)
    kth_precision=np.array(kth_precision)    
    
    average_accuracies = np.mean(kth_accuracies)
    average_f1_scores = np.mean(kth_f1_scores)
    average_recalls= np.mean(kth_recall)
    average_precision = np.mean(kth_precision)

    accuracies.append(average_accuracies)
    f1_scores.append(average_f1_scores)
    recalls.append(average_recalls)
    precisions.append(average_precision)



    

Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
Extracting EDF parameters from d:\porojects\EEG classification\datasets\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Epoch 1/100
49/49 [==============================] - 18s 119ms/step - loss: 6.7546 - accuracy: 0.2919 - val_loss: 5.8570 - val_accuracy: 0.3292
Epoch 2/100
49/49 [==============================] - 4s 91ms/step - loss: 5.9654 - accuracy: 0.4224 - val_loss: 5.4964 - val_accuracy: 0.5217
Epoch 3/100
49/49 [==============================] - 4s 91ms/step - loss: 5.4385 - accuracy: 0.5452 - val_loss: 5.1935 - val_accuracy: 0.5839
Epoch 4/100
49/49 [==============================] - 4s 91ms/step - loss: 4.9725 - accuracy: 0.6356 - val_loss: 4.8008 - val_accuracy: 0.6770
Epoch 5/100
49/49 [==============================] - 5s 94ms/step - loss: 4.5688 - accuracy: 0.7198 - val_loss: 4.4716 - val_accuracy: 0.7267
Epoch 6/100
49/49 [==============================] - 5s 94ms/step - loss: 4.2539 - accuracy: 0.7709 - val_loss: 4.1566 - val_accuracy: 0.7702
Epoch 7/100
49/49 [==============================] - 5s 94ms/step - loss: 3.9596 - accuracy: 0.8088 - val_loss: 3.9525 - val_accuracy: 0.7888
Epoc

...

Epoch 1/100
49/49 [==============================] - 20s 189ms/step - loss: 6.5974 - accuracy: 0.3409 - val_loss: 5.6971 - val_accuracy: 0.5590
Epoch 2/100
49/49 [==============================] - 8s 169ms/step - loss: 5.6967 - accuracy: 0.5190 - val_loss: 5.3219 - val_accuracy: 0.6460
Epoch 3/100
49/49 [==============================] - 8s 168ms/step - loss: 5.1407 - accuracy: 0.6522 - val_loss: 4.8887 - val_accuracy: 0.7391
Epoch 4/100
49/49 [==============================] - 8s 168ms/step - loss: 4.7020 - accuracy: 0.7474 - val_loss: 4.5209 - val_accuracy: 0.7702
Epoch 5/100
49/49 [==============================] - 8s 171ms/step - loss: 4.3703 - accuracy: 0.7723 - val_loss: 4.1747 - val_accuracy: 0.8199
Epoch 6/100
49/49 [==============================] - 8s 172ms/step - loss: 3.9831 - accuracy: 0.8544 - val_loss: 3.8920 - val_accuracy: 0.8634
Epoch 7/100
49/49 [==============================] - 8s 171ms/step - loss: 3.7101 - accuracy: 0.8640 - val_loss: 3.6153 - val_accuracy: 0.888

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Epoch 1/100
49/49 [==============================] - 20s 187ms/step - loss: 6.7528 - accuracy: 0.3078 - val_loss: 5.7993 - val_accuracy: 0.4907
Epoch 2/100
49/49 [==============================] - 8s 166ms/step - loss: 5.9057 - accuracy: 0.4865 - val_loss: 5.4435 - val_accuracy: 0.6273
Epoch 3/100
49/49 [==============================] - 8s 166ms/step - loss: 5.3757 - accuracy: 0.5728 - val_loss: 5.1000 - val_accuracy: 0.6584
Epoch 4/100
49/49 [==============================] - 8s 165ms/step - loss: 5.0593 - accuracy: 0.6384 - val_loss: 4.8053 - val_accuracy: 0.7019
Epoch 5/100
49/49 [==============================] - 8s 166ms/step - loss: 4.6352 - accuracy: 0.7026 - val_loss: 4.7468 - val_accuracy: 0.6087
Epoch 6/100
49/49 [==============================] - 8s 165ms/step - loss: 4.3006 - accuracy: 0.7654 - val_loss: 4.2127 - val_accuracy: 0.7950
Epoch 7/100
49/49 [==============================] - 8s 165ms/step - loss: 3.9858 - accuracy: 0.8261 - val_loss: 3.9363 - val_accuracy: 0.813

C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
C:\Users\nima8\AppData\Local\Programs\Python\Python38\lib\contextlib.py:120: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Epoch 1/100
49/49 [==============================] - 21s 195ms/step - loss: 6.7244 - accuracy: 0.3195 - val_loss: 5.7529 - val_accuracy: 0.3851
Epoch 2/100
49/49 [==============================] - 9s 174ms/step - loss: 5.8104 - accuracy: 0.4921 - val_loss: 5.3441 - val_accuracy: 0.6957
Epoch 3/100
49/49 [==============================] - 8s 172ms/step - loss: 5.3379 - accuracy: 0.6011 - val_loss: 4.9722 - val_accuracy: 0.7702
Epoch 4/100
49/49 [==============================] - 8s 172ms/step - loss: 4.8658 - accuracy: 0.7026 - val_loss: 4.5821 - val_accuracy: 0.8385
Epoch 5/100
49/49 [==============================] - 8s 172ms/step - loss: 4.4768 - accuracy: 0.7640 - val_loss: 4.3174 - val_accuracy: 0.8261
Epoch 6/100
49/49 [==============================] - 8s 171ms/step - loss: 4.1553 - accuracy: 0.8102 - val_loss: 4.0252 - val_accuracy: 0.8571
Epoch 7/100
49/49 [==============================] - 8s 170ms/step - loss: 3.8818 - accuracy: 0.8413 - val_loss: 3.7528 - val_accuracy: 0.869

In [5]:
print(accuracies)
print("------------------------------")
print(f1_scores)
print("------------------------------")
print(recalls)
print("------------------------------")
print(precisions)

[0.6975801572897762, 0.5386569872958258, 0.7360556563823352, 0.5552934059286146, 0.7295220810647308, 0.5695704779189353, 0.7431941923774954, 0.7186932849364791, 0.7049001814882032]
------------------------------
[0.6850931308585262, 0.5399392835708469, 0.7331237234353948, 0.552146561080057, 0.7298359779654103, 0.57253107017706, 0.7419463837586469, 0.7242441591844129, 0.6912684430268266]
------------------------------
[0.6975801572897762, 0.5386569872958258, 0.7360556563823352, 0.5552934059286146, 0.7295220810647308, 0.5695704779189353, 0.7431941923774954, 0.7186932849364791, 0.7049001814882032]
------------------------------
[0.7105121158451999, 0.5749593476352592, 0.7879994559045216, 0.6012141623994057, 0.7593926996932228, 0.6200486415592639, 0.7704381382960508, 0.7685292378485448, 0.7455832671296532]
